In [38]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from fastapi import FastAPI
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
import joblib
from pydantic import BaseModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.impute import SimpleImputer

In [2]:
new_file = pd.read_csv(r'C:\Users\UltraBook 3.1\Desktop\data_analysis projects\Olame_projects\movie_recommendation\ml-32m\merged_data1.csv')

In [ ]:
# new_file.isnull().sum()
new_file.drop_duplicates(inplace=True)

In [5]:
new_file.isnull().sum()

rating        0
tag       58248
genres        0
mean          0
count         0
dtype: int64

In [ ]:
new_file['genres'] = new_file['genres'].apply(lambda x: x.split('|'))

In [ ]:
# replace missing tags with unknown
new_file.tag = new_file.tag.fillna('Unknown')

In [8]:
len(new_file.tag.unique())

140980

In [9]:
new_file.describe().T

,count,mean,std,min,25%,50%,75%,max
rating,2110334.0,3.331239,0.957122,0.500000,3.000000,3.500000,3.500000,5.000000
mean,2110334.0,3.480477,0.151095,1.641463,3.435034,3.495455,3.545565,4.040262
count,2110334.0,19321.512665,34003.187871,33.000000,1049.000000,7031.000000,21929.000000,318459.000000


In [ ]:
mlb = MultiLabelBinarizer()
genre_encoded = mlb.fit_transform(new_file['genres'])
# tag_encoded = mlb.fit_transform(new_file['tag'])
# Create a DataFrame with multi-hot encoded genres
genre_df = pd.DataFrame(genre_encoded, columns=mlb.classes_)
# tag_df = pd.DataFrame(tag_encoded, columns=mlb.classes_)

In [11]:
# new_file
genre_df.index = new_file.index

In [12]:
# # Concatenate with original DataFrame
df_final = pd.merge(new_file, genre_df, left_index=True, right_index=True)

In [13]:
# use frequency scoring for the tags
tag_counts = df_final['tag'].value_counts()
df_final['Tag_score'] = df_final['tag'].map(tag_counts)

In [14]:
# drop the tag column and use the tag_score for clustering
df_final.drop(columns=['genres', 'tag'], inplace=True)

In [126]:
# select some data before training for prediction/recommendation
testing_data = new_file.sample(n=4)
# drop from the original data
df_final.drop(testing_data.index, inplace=True)

In [53]:
testing_data.reset_index(inplace=True)
testing_data.drop(columns=['index'], inplace=True)

In [127]:
testing_data

,rating,tag,genres,mean,count
1436554,3.5,terrorist,Action|Drama|Horror,3.538224,37280
2529063,3.5,fake New York settings,Action|Drama,3.509797,2756
84369458,0.5,wilderness,Action|Adventure|Thriller,3.493333,225
2971538,3.5,genre:Sci-Fi Adventure,Action|Sci-Fi,3.457444,8906


In [118]:
df_final

,rating,mean,count,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Tag_score
0,2.5,3.754257,109346,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,2200
1,3.0,3.754257,109346,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,2200
2,4.0,3.754257,109346,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,2200
4,5.0,3.754257,109346,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,2200
11,1.0,3.754257,109346,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,2200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96897571,0.5,3.411765,34,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,58248
96898163,1.0,3.400000,35,0,0,0,0,0,1,1,...,0,0,0,1,0,0,0,0,0,58248
96898316,1.5,3.440476,42,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,58248
96898317,0.5,3.440476,42,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,58248


In [17]:
columns_sure = df_final.columns

In [81]:
# initialiing scaler
standardizer = StandardScaler()

# implement
scaled_values = standardizer.fit_transform(df_final)
scaled_values = pd.DataFrame(scaled_values)
scaled_values.columns =columns_sure
scaled_values

,rating,mean,count,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Tag_score
0,-0.868478,1.811977,2.647537,-0.117759,-0.561686,2.265100,4.135301,4.398853,1.617279,-0.377416,...,-0.424344,-0.167437,-0.146929,-0.290635,-0.382385,-0.453476,-0.572604,-0.204227,-0.127359,-0.005139
1,-0.346078,1.811977,2.647537,-0.117759,-0.561686,2.265100,4.135301,4.398853,1.617279,-0.377416,...,-0.424344,-0.167437,-0.146929,-0.290635,-0.382385,-0.453476,-0.572604,-0.204227,-0.127359,-0.005139
2,0.698721,1.811977,2.647537,-0.117759,-0.561686,2.265100,4.135301,4.398853,1.617279,-0.377416,...,-0.424344,-0.167437,-0.146929,-0.290635,-0.382385,-0.453476,-0.572604,-0.204227,-0.127359,-0.005139
3,1.743520,1.811977,2.647537,-0.117759,-0.561686,2.265100,4.135301,4.398853,1.617279,-0.377416,...,-0.424344,-0.167437,-0.146929,-0.290635,-0.382385,-0.453476,-0.572604,-0.204227,-0.127359,-0.005139
4,-2.435677,1.811977,2.647537,-0.117759,-0.561686,2.265100,4.135301,4.398853,1.617279,-0.377416,...,-0.424344,-0.167437,-0.146929,-0.290635,-0.382385,-0.453476,-0.572604,-0.204227,-0.127359,-0.005139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2110321,-2.958077,-0.454765,-0.567227,-0.117759,1.780355,2.265100,-0.241820,-0.227332,-0.618323,-0.377416,...,2.356578,-0.167437,-0.146929,-0.290635,-0.382385,-0.453476,-0.572604,-0.204227,-0.127359,5.868082
2110322,-2.435677,-0.532629,-0.567197,-0.117759,-0.561686,-0.441482,-0.241820,-0.227332,1.617279,2.649597,...,-0.424344,-0.167437,-0.146929,3.440744,-0.382385,-0.453476,-0.572604,-0.204227,-0.127359,5.868082
2110323,-1.913278,-0.264742,-0.566991,-0.117759,1.780355,2.265100,4.135301,-0.227332,-0.618323,-0.377416,...,-0.424344,-0.167437,-0.146929,-0.290635,-0.382385,-0.453476,-0.572604,-0.204227,-0.127359,5.868082
2110324,-2.958077,-0.264742,-0.566991,-0.117759,1.780355,2.265100,4.135301,-0.227332,-0.618323,-0.377416,...,-0.424344,-0.167437,-0.146929,-0.290635,-0.382385,-0.453476,-0.572604,-0.204227,-0.127359,5.868082


In [56]:
X_scaled = standardizer.fit_transform(scaled_values)
# Apply KMeans clustering with 4 clusters.
kmeans = KMeans(n_clusters=5, random_state=42)
scaled_values['cluster'] = kmeans.fit_predict(X_scaled)

In [21]:
# just the cluster series
cluster_grp = scaled_values.cluster
cluster_grp.index = df_final.index

In [22]:
final_merge = pd.merge(df_final,cluster_grp, left_index=True, right_index= True)

In [23]:
final_merge

,rating,mean,count,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Tag_score,cluster
0,2.5,3.754257,109346,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,2200,3
1,3.0,3.754257,109346,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,2200,3
2,4.0,3.754257,109346,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,2200,3
4,5.0,3.754257,109346,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,2200,3
11,1.0,3.754257,109346,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,2200,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96897571,0.5,3.411765,34,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,58248,4
96898163,1.0,3.400000,35,0,0,0,0,0,1,1,...,0,0,1,0,0,0,0,0,58248,2
96898316,1.5,3.440476,42,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,58248,3
96898317,0.5,3.440476,42,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,58248,3


In [79]:
new_file

,rating,tag,genres,mean,count
0,2.5,imdb top 250,Adventure|Animation|Children|Comedy|Fantasy,3.754257,109346
1,3.0,imdb top 250,Adventure|Animation|Children|Comedy|Fantasy,3.754257,109346
2,4.0,imdb top 250,Adventure|Animation|Children|Comedy|Fantasy,3.754257,109346
3,3.0,imdb top 250,Adventure|Animation|Children|Comedy|Fantasy,3.754257,109346
4,5.0,imdb top 250,Adventure|Animation|Children|Comedy|Fantasy,3.754257,109346
...,...,...,...,...,...
98937255,3.5,NaN,Drama,3.514706,34
98937256,3.5,NaN,Comedy|Drama,3.441176,34
98937257,3.5,NaN,Drama,3.514706,34
98937258,3.5,NaN,Drama,3.426471,34


In [ ]:
scaled_values.cluster.value_counts()

In [54]:
# build the model for deployment

class DropDuplicatesTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X.drop_duplicates(inplace=True)
        return X


class GenresSplitterTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()  # avoid changing the original DataFrame
        X['genres'] = X['genres'].str.split('|')
        return X


class TagVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, max_features=300):
        self.vectorizer = TfidfVectorizer(max_features=max_features)

    def fit(self, X, y=None):
        self.vectorizer.fit(X['tag'].fillna('Unknown'))
        return self

    def transform(self, X):
        X = X.copy()
        tag_matrix = self.vectorizer.transform(X['tag'].fillna('Unknown'))
    
        # Calculate the mean TF-IDF score for each tag
        avg_tfidf_scores = tag_matrix.mean(axis=1)
        tag_mean_tfidf = pd.Series([float(score) for score in avg_tfidf_scores], index=X.index)
    
        # Assign to the DataFrame
        X = X.assign(tag_mean_tfidf=tag_mean_tfidf)
    
        # Optionally drop the original tag column
        X.drop(columns=['tag'], inplace=True)
        return X
        

class GenresMultiLabelBinarizer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.mlb = MultiLabelBinarizer()

    def fit(self, X, y=None):
        self.mlb.fit(X['genres'])  # X is expected to be a DataFrame
        return self

    def transform(self, X):
        genre_encoded = self.mlb.transform(X['genres'])
        genre_df = pd.DataFrame(genre_encoded, columns=self.mlb.classes_, index=X.index)
        # Drop the original genre column and concat the new one-hot encoded columns
        X = pd.merge(X, genre_df, left_index=True, right_index=True)
        X.drop('genres', axis=1, inplace=True)
        return X


class TagFrequencyScorer(BaseEstimator, TransformerMixin):
    def __init__(self, tag_column='tag', score_column='Tag_score'):
        self.tag_column = tag_column
        self.score_column = score_column
        self.tag_counts_ = None

    def fit(self, X, y=None):
        self.tag_counts_ = X[self.tag_column].value_counts()
        return self

    def transform(self, X):
        X = X.copy()
        X[self.score_column] = X[self.tag_column].map(self.tag_counts_)
        return X



class KMeansClusteringTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, n_clusters=5, random_state=42):
        self.n_clusters = n_clusters
        self.random_state = random_state
        self.scaler = StandardScaler()
        self.kmeans = KMeans(n_clusters=self.n_clusters, random_state=self.random_state)

    def fit(self, X, y=None):
        X_scaled = self.scaler.fit_transform(X)
        self.kmeans.fit(X_scaled)
        return self

    def transform(self, X):
        X_scaled = self.scaler.transform(X)
        clusters = self.kmeans.predict(X_scaled)
        
        # Convert X to DataFrame (in case it's still an array)
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)

        X_transformed = X.copy()
        X_transformed['cluster'] = clusters
        return X_transformed



In [53]:
drop_duplicates = DropDuplicatesTransformer().fit_transform(new_file)

In [19]:
split = GenresSplitterTransformer().fit_transform(drop_duplicates)

In [29]:
binarizing = GenresMultiLabelBinarizer()
output = binarizing.fit_transform(X=split)

In [27]:
frequent = TagFrequencyScorer(tag_column='tag')

In [31]:
freq = frequent.fit_transform(output)

In [33]:
vec = TagVectorizer()
vector = vec.fit_transform(freq)

C:\Users\UltraBook 3.1\AppData\Local\Temp\ipykernel_23248\2283103398.py:36: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  tag_mean_tfidf = pd.Series([float(score) for score in avg_tfidf_scores], index=X.index)


In [35]:
vector.isnull().sum()

rating                    0
mean                      0
count                     0
(no genres listed)        0
Action                    0
Adventure                 0
Animation                 0
Children                  0
Comedy                    0
Crime                     0
Documentary               0
Drama                     0
Fantasy                   0
Film-Noir                 0
Horror                    0
IMAX                      0
Musical                   0
Mystery                   0
Romance                   0
Sci-Fi                    0
Thriller                  0
War                       0
Western                   0
Tag_score             58248
tag_mean_tfidf            0
dtype: int64

In [11]:
# API
# Define FastAPI app
app = FastAPI(
    title="Movie Cluster API",
    version="1.0.0"
)

In [55]:
complete_pipeline = Pipeline([
    ('drop_duplicates', DropDuplicatesTransformer()),
    ('split_genres', GenresSplitterTransformer()),
    ('encode_genres', GenresMultiLabelBinarizer()),
    ('tag_score', TagFrequencyScorer(tag_column='tag')),
    ('fill_vectorize_tags', TagVectorizer()),
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('kmeans', KMeansClusteringTransformer(n_clusters=5, random_state=42))
])

In [56]:
df_with_clusters = complete_pipeline.fit_transform(new_file)

C:\Users\UltraBook 3.1\AppData\Local\Temp\ipykernel_23248\1742894205.py:36: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  tag_mean_tfidf = pd.Series([float(score) for score in avg_tfidf_scores], index=X.index)


In [57]:
df_with_clusters

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,cluster
0,2.5,3.754257,109346.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2200.0,0.005770,0
1,3.0,3.754257,109346.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2200.0,0.005770,0
2,4.0,3.754257,109346.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2200.0,0.005770,0
3,5.0,3.754257,109346.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2200.0,0.005770,0
4,1.0,3.754257,109346.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2200.0,0.005770,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2110329,0.5,3.411765,34.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003333,2
2110330,1.0,3.400000,35.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003333,4
2110331,1.5,3.440476,42.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003333,2
2110332,0.5,3.440476,42.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003333,2


In [42]:
# Save the pipeline

# Filename with version
version = "1.0.0"
file_directory =rf"C:\Users\UltraBook 3.1\Desktop\data_analysis projects\Olame_projects\movie_recommendation\cluster_pipeline_v{version}.pkl"
joblib.dump(complete_pipeline, file_directory)

print(f"Pipeline saved to: {file_directory}")

Pipeline saved to: C:\Users\UltraBook 3.1\Desktop\data_analysis projects\Olame_projects\movie_recommendation\cluster_pipeline_v1.0.0.pkl


In [9]:
# Load the trained pipeline
file_directory = r"C:\Users\UltraBook 3.1\Desktop\data_analysis projects\Olame_projects\movie_recommendation\cluster_pipeline_v1.0.0.pkl"
pipeline = joblib.load(file_directory)

# Example usage
# data should be a DataFrame with the same columns as during training
prediction = pipeline.transform(testing_data)

NameError: name 'testing_data' is not defined

In [129]:
prediction

,rating,mean,count,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Tag_score,cluster
1436554,3.5,3.538224,37280,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,490,0
2529063,3.5,3.509797,2756,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
84369458,0.5,3.493333,225,0,1,1,0,0,0,0,...,0,0,0,0,0,1,0,0,640,2
2971538,3.5,3.457444,8906,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,2,2


In [12]:

# Load the trained pipeline
file_directory = r"C:\Users\UltraBook 3.1\Desktop\data_analysis projects\Olame_projects\movie_recommendation\cluster_pipeline_v1.0.0.pkl"
pipeline = joblib.load(file_directory)


# Input model for prediction
class MovieData(BaseModel):
    tag: str
    genres: str
    rating: float
    mean: float
    count: int

@app.post("/predict_cluster")
def predict_cluster(data: MovieData):
    # Convert input to DataFrame
    input_df = pd.DataFrame([data.dict()])

    # Transform and predict using the pipeline
    transformed = pipeline.transform(input_df)
    cluster = transformed['cluster'].iloc[0]

    return {
        "cluster": int(cluster),
        "status": "success"
    }


In [ ]:
uvicorn main:app --reload
